In [1]:
import tensorflow as tf

In [2]:
mnist = tf.keras.datasets.mnist

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [4]:
x_train, x_test = x_train / 255.0, x_test / 255.0

In [5]:
mlp_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(128,activation = "relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])

In [6]:
mlp_model.compile(optimizer="adam", loss = "sparse_categorical_crossentropy", metrics=["accuracy"])

In [7]:
mlp_model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2682 - accuracy: 0.9233
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1157 - accuracy: 0.9658
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0804 - accuracy: 0.9758
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0586 - accuracy: 0.9814
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0456 - accuracy: 0.9857


In [8]:
converter = tf.lite.TFLiteConverter.from_keras_model(mlp_model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\Public\Documents\ESTsoft\CreatorTemp\tmpjg_kmqgx\assets


INFO:tensorflow:Assets written to: C:\Users\Public\Documents\ESTsoft\CreatorTemp\tmpjg_kmqgx\assets


In [9]:
with open("./mnist.tflite", "wb") as f:
    f.write(tflite_model)

In [10]:
mlp_model.save("./mlp_model2/")

INFO:tensorflow:Assets written to: ./mlp_model2/assets


INFO:tensorflow:Assets written to: ./mlp_model2/assets


In [11]:
saved_model = tf.keras.models.load_model("./mlp_model2/")

In [12]:
converter = tf .lite.TFLiteConverter.from_saved_model("./mlp_model2/")

In [13]:
tflite_model = converter.convert()

In [14]:
with open("./saved_model.tflite", "wb") as f:
    f.write(tflite_model)

In [15]:
class Inc(tf.keras.layers.Layer):
    def call(self, inputs):
        return inputs +1

inc = Inc()

In [16]:
class Inc_Graph(tf.keras.layers.Layer):
    @tf.function
    def call(self, inputs):
        return inputs + 1

inc_g = Inc_Graph()

In [17]:
inc_g2 = tf.function(inc)

In [18]:
print(inc_g(tf.constant(3)))
print(inc_g(tf.constant([3,2])))
print(inc_g(tf.constant([[3,2],[1.0,5.0]])))

print(inc_g2(tf.constant(3)))
print(inc_g2(tf.constant([3,2])))
print(inc_g2(tf.constant([[3,2],[1.0,5.0]])))

tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor([4 3], shape=(2,), dtype=int32)
tf.Tensor(
[[4. 3.]
 [2. 6.]], shape=(2, 2), dtype=float32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor([4 3], shape=(2,), dtype=int32)
tf.Tensor(
[[4. 3.]
 [2. 6.]], shape=(2, 2), dtype=float32)


In [19]:
print(inc_g(tf.constant(4)))
print(inc_g2(tf.constant(4)))

tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)


In [20]:
concrete_fun = inc_g.call.get_concrete_function(tf.TensorSpec(shape=(1,3), dtype=tf.float32))
print(concrete_fun(tf.constant([[1.0,2.0,3.0]])))

tf.Tensor([[2. 3. 4.]], shape=(1, 3), dtype=float32)


In [21]:
concrete_fun = inc_g2.get_concrete_function(tf.TensorSpec(shape=(1,3), dtype=tf.float32))
print(concrete_fun(tf.constant([[1.0,2.0,3.0]])))

tf.Tensor([[2. 3. 4.]], shape=(1, 3), dtype=float32)


In [22]:
mlp_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])

In [23]:
graph_model = tf.function(mlp_model)
concrete_func = graph_model.get_concrete_function(
    tf.TensorSpec(shape=mlp_model.inputs[0].shape, dtype=mlp_model.inputs[0].dtype))

In [24]:
# class MLP_Model(tf.keras.Model):
#     def __init__(self):
#         super(MLP_Model, self).__init__()
#         self.flatten = tf.keras.layers.Flatten()
#         self.dense = tf.keras.layers.Dense(128, activation="relu")
#         self.softmax = tf.keras.layers.Dense(10, activation="softmax")

#     @tf.function
#     def call(self, inputs):
#         x = self.flatten(inputs)
#         x = self.dense(x)
#         return self.softmax(x)

# mlp_model = MLP_Model()
# concrete_func = mlp_model.call.get_concrete_function(
#     tf.TensorSpec(shape=mlp_model.inputs[0].shape, dtype=mlp_model.inputs[0].dtype))